<a href="https://colab.research.google.com/github/ainhoaVivel/Boundless/blob/main/P1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
import os
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
os.chdir('/content/gdrive/My Drive/AIF_P1_data') ## mount Google Drive path where files are saved
!ls

Mounted at /content/gdrive
Doc		map.txt				  __pycache__
helpers_mod.py	P1_Barreiro_Gonzalez_Vivel.ipynb  search_mod.py


In [71]:
def load_data(name="map.txt"):
  file = open(name, mode="r")

  #We read the header what to hope in the rest of the file
  header = file.readline()
  header.split(' ')
  map_dims = [] #Features of the map
  for item in header:
    if item.isnumeric(): #The file may contains encode character that we should avoid
      map_dims.append(int(item))
  map_dims

  #Map read process
  mapa=[]
  for i in range(map_dims[0]):
    line = file.readline().split(" ")
    mapa.append(list(map(int, line)))

  #Intial satate
  line = file.readline().replace("\n","").split(" ")
  init_state = list(map(int, line))

  #Goal state
  line = file.readline().replace("\n","").split(" ")
  goal_state = list(map(int, line))

  return map_dims, mapa, init_state, goal_state

In [ ]:
print(load_data())

([6, 8], [[1, 2, 2, 1, 1, 3, 2, 1], [3, 2, 5, 1, 2, 4, 2, 1], [2, 3, 5, 3, 3, 2, 4, 4], [1, 3, 1, 1, 4, 1, 1, 2], [3, 5, 1, 2, 4, 4, 1, 4], [1, 3, 1, 1, 4, 1, 1, 1]], [0, 4, 0], [5, 5, 8])


In [ ]:
!ls

Doc		map.txt				  __pycache__
helpers_mod.py	P1_Barreiro_Gonzalez_Vivel.ipynb  search_mod.py


El libro de consulta de referencia para esta práctica es el "Artificial Intelligence: A Modern Approach." (conocido como AIMA http://aima.cs.berkeley.edu/) de Russell y Norvig. En particular, el capítulo 3 ofrece una panorámica completa sobre técnicas clásicas de resolución de problemas basadas en búsqueda en espacios de estados. 

El libro viene con un conjunto de módulos Python que contiene muchos de los algoritmos implementados. Es por ello, que en la medida de lo posible utilizaremos sus implementaciones, que originalmente pueden consultarse en el repositorio de AIMA-Python (https://github.com/aimacode/aima-python), y de los que hemos incorporado una copia adaptada a las necesidades de nuestra Asignatura en este boletín.

Para trabajar con los algoritmos de búsqueda relevantes, primero nos familiarizaremos con las clases y funciones clave que nos permitirá resolver problemas mediante estrategias de búsqueda vistas en las sesiones expositiva. Será entonces que trabajaremos con un problema concreto, el 8-puzzle, realizando pequeñas modificaciones al código existente para incorporar y estudiar el comportamiento de las heurísticas. Finalmente, se os pedirá que implementéis completamente una solución al problema del Viajante de Comercio y estudieis las heurísticas que implementeis. 

¡Empecemos!

En primer lugar, empecemos por importar los módulos python especialmente preparados para esta Práctica 1.

In [ ]:
# contiene algoritmos de búsqueda implementado en AIMA
from search_mod import *

# algunas funciones auxiliares
from helpers_mod import *

- Sublcase de problema. 
    - Rperesentación internas: 
        - mapa: que será unha matriz como a obtida da entrada. 
        - mapa_dims: que serán as dimensións do mapa. 
        - initial_state:
        - goal_sate:
    - actions
    - is_goal
    - action_cost 

- $state := (x, y, o)$
  Sendo x a posición na fila y na columna e o a orientación. O rango de X e Y ven dado polo caso. O rango de $o \in [0, 1, 2, 3]$. Na especificación contemplamos que se poda introducir outro valor de orientación que tomaremos sempre como indefinido. 


# New Section

In [77]:
from enum import Enum, IntEnum, auto

class Action(Enum):
  TURN_LEFT = auto()
  TURN_RIGHT = auto()
  WALK = auto()

class Orientation(IntEnum):
  NORTH = 0
  EAST = 1
  SOUTH = 2
  WEST = 3
  UNDEFINED = 8

class PathProblem(Problem):
  def __init__(self, map, dims, initial, goal=(0, 0, 8)):
    self.map = map
    self.dims = dims
    self.initial, self.goal = initial, goal

  def in_bounds(self, pos):
    return 0 <= pos[0] < self.dims[0] and 0 <= pos[1] < self.dims[1]
  
  def next_pos(self, state):
    if state[2] == Orientation.NORTH:
      return (state[0], state[1] - 1)
    elif state[2] == Orientation.EAST:
      return (state[0] + 1, state[1])
    elif state[2] == Orientation.SOUTH:
      return (state[0], state[1] + 1)
    elif state[2] == Orientation.WEST:
      return (state[0] - 1, state[1])
    else:
      raise ValueError("Undefined orientation")
  
  def actions(self, state):
    moves = [Action.TURN_LEFT, Action.TURN_RIGHT]
    if self.in_bounds(self.next_pos(state)):
      moves.append(Action.WALK)
    return moves

  def result(self, state, action):
    s = list(state)
    if action == Action.TURN_LEFT:
      s[2] = (s[2] - 1) % 4
    elif action == Action.TURN_RIGHT:
      s[2] = (s[2] + 1) % 4
    elif action == Action.WALK:
      s[0:2] = self.next_pos(s)
    else:
      raise ValueError("Invalid action")
    return tuple(s)

  def is_goal(self, state):
    return state[0:2] == self.goal[0:2]

  def action_cost(self, s, a, s1):
    if a == Action.WALK:
      return self.map[s1[0]][s1[1]]
    else:
      return 1

  def h(self, node):
    # Manhattan heuristic
    return abs(self.goal[0] - node.state[0]) + abs(self.goal[1] - node.state[1])
  
  def h_euclidian(self, node):
    # Euclidian heuristic
    return pow(pow(self.goal[0] - node.state[0], 2) + pow(self.goal[0] - node.state[0], 2), 1/2)
  
  def h_chebyshev(self, node):
    diffs = []
    diffs.append(abs(self.goal[0] - node.state[0]))
    diffs.append(abs(self.goal[1] - node.state[1]))
    return max(diffs)

In [78]:
def path_problem_from_file(name="map.txt"):
  map_dims, map, init_state, goal_state = load_data(name)
  return PathProblem(map, map_dims, tuple(init_state), tuple(goal_state))


In [79]:
problem = path_problem_from_file()

TypeError: ignored

In [81]:
sol =  astar_search(problem)

In [168]:
print("Nodo solución: ", sol)
print (f'Coste del camino: {sol.path_cost:f}')
print (f'Lista de acciones: {path_actions(sol)}')
print (f'Lista de estados:')
print (path_states(sol))

Nodo solución:  <(7, 4, 1)>
Coste del camino: 13.000000
Lista de acciones: [<Action.TURN_LEFT: 1>, <Action.WALK: 3>, <Action.WALK: 3>, <Action.WALK: 3>, <Action.TURN_RIGHT: 2>, <Action.WALK: 3>]
Lista de estados:
[(6, 7, 1), (6, 7, 0), (6, 6, 0), (6, 5, 0), (6, 4, 0), (6, 4, 1), (7, 4, 1)]


In [183]:
def print_state(depth, cost, action, state):
  print("[",depth, cost, action,"(",state[0],",",state[1],")",Orientation(state[2]).name,"]")

def print_path(sol):
  states=path_states(sol)
  #Initial state
  depth=0
  print_state(depth,0,"ini",states[0])
  states.pop(0)
  for action, state in zip(path_actions(sol), states):
    print(action.name)
    depth+=1
    print_state(depth,0,action.name,state)


In [184]:
print_path(sol)

[ 0 0 ini ( 6 , 7 ) EAST ]
TURN_LEFT
[ 1 0 TURN_LEFT ( 6 , 7 ) NORTH ]
WALK
[ 2 0 WALK ( 6 , 6 ) NORTH ]
WALK
[ 3 0 WALK ( 6 , 5 ) NORTH ]
WALK
[ 4 0 WALK ( 6 , 4 ) NORTH ]
TURN_RIGHT
[ 5 0 TURN_RIGHT ( 6 , 4 ) EAST ]
WALK
[ 6 0 WALK ( 7 , 4 ) EAST ]


## Functions to generate aleatory maps and positions

In [102]:
import random


def gen_map(n, m):
  map = []
  for i in range(n):
    map_line = []
    for j in range(m):
      map_line.append(random.randint(1, 5))
    map.append(map_line)
  return map, (n, m)

gen_map(3, 3) 

def gen_position(n, m):
  return (random.randint(0, n-1), random.randint(0, m-1), random.randint(0, 3))

gen_position(3,3)

(2, 2, 3)

## Tests

In [72]:
map_sizes = [3,4,5,6,7,8,9]
n_maps = 4
n_positions = 5

In [73]:
pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [106]:
from tqdm.notebook import tqdm
from numpy import mean

costs = []

for rep in tqdm(range(100)):
  for size in map_sizes:
    for i in range(n_maps):
      map, map_dims = gen_map(size, size)
      for i in range(n_positions):
        init_pos = gen_position(size, size)
        goal_pos = gen_position(size, size)
        problem = PathProblem(map, map_dims, init_pos, goal_pos)
        sol =  astar_search(problem)
        costs.append(sol.path_cost)

print(mean(costs))


  0%|          | 0/100 [00:00<?, ?it/s]

11.77207142857143
